## Demonstration on how to use Microsoft's open-source resource estimator with Qualtran Bloqs

Having interopability with Microsoft's open-source resource estimator is useful because it allows users more flexiblity with their Qualtran programs and allows users to verify estimates given by Qualtran's resource estimator tooling.

In this notebook, we will demonstrate doing resource estimation on the `ModExp` Bloq, the essential subroutine in Shor's factoring algorithm.

In [ ]:
from qualtran.bloqs.factoring.mod_exp import ModExp
from qualtran.drawing import show_bloq

N = 13*17 # integer to factor
g = 8

mod_exp = ModExp(base=g, mod=N, exp_bitsize=32, x_bitsize=32)
show_bloq(mod_exp)

In the next cell we have a function which takes in a Bloq and returns LogicalCounts, a python object which stores information about the logical data and gate counts. You can initialize the LogicalCounts object with a dictionary with information about your quantum algorithm. You can populate fields such as `numQubits` and `tCount` with information from a Qualtran `Bloq` as show below. For a more complete overview of the fields which you can specify in LogicalCounts, click [here](https://learn.microsoft.com/en-us/azure/quantum/resource-estimator-known-estimates?tabs=tabid-known-estimates-qsharp) and look for the table which specifies the available functions with AccountForEstimates.

In [ ]:
def get_bloq_logical_counts(bloq):
    from qsharp.estimator import LogicalCounts
    from qualtran._infra.gate_with_registers import get_named_qubits

    # This way of calculating the number of qubits could be inaccurate if a sub_bloq
    # allocates new qubits but this should be a good estimate in most cases.
    num_qubits = sum(len(r) for r in get_named_qubits(bloq.signature).values())
    complexity = bloq.t_complexity()

    return LogicalCounts({
        "numQubits": num_qubits,
        "tCount": complexity.t,
        "rotationCount": complexity.rotations,
        "rotationDepth": complexity.rotations
    })

The LogicalCounts object has the key method, `estimate`, which returns an `EstimatorResult` which contains the physical resource estimates. By running the cell below, you can see a nice display of the physical resource estimates including runtime, physical qubits, and many more important resource estimates. For a more complete description of the `EstimatorResult`, click [here](https://learn.microsoft.com/en-us/azure/quantum/overview-resource-estimator-output-data).

In [ ]:
estimates = get_bloq_logical_counts(mod_exp).estimate([
    {"qubitParams": {"name": "qubit_gate_ns_e3"}},
    {"qubitParams": {"name": "qubit_gate_ns_e4"}},
    {"qubitParams": {"name": "qubit_maj_ns_e4"}, "surfaceCode": {"name": "floquet_code"}},
    {"qubitParams": {"name": "qubit_maj_ns_e6"}, "surfaceCode": {"name": "floquet_code"}},
])

estimates